In [1]:
# Import all neccesary python libraries for analysis.
import pymc as pm
import pandas as pd
import pyreadr
import nutpie
import arviz as az

# Overview

The following analysis is a Bayesian hierchial model generated using the PyMC PPL framework and the associated Rust based nutpie Hamiltonian monte Carlo sampler. The basis for this analysis is from Harrer et al. (2021) whom used R and the brms package.

In [2]:
# Import rda file into OrderedDict.
keys = pyreadr.read_r("data/ThirdWave.rda")

# Extract from OrderedDict the pandas dataframe.
df = keys["ThirdWave"]

# Extract the data for analysi
seTE = df["seTE"]
# Convert to integer to make the model code work below.


df['Author'] = pd.Categorical(df['Author'])
author_id = df['Author'].cat.codes
TE = df["TE"]

# The Bayesian Hierachial model

\begin{align*}  

\ \hat\theta_k &\sim Normal(\theta_k, \sigma_k)
\\ \theta_k &\sim Normal(\mu, \tau) 
\\ \mu &\sim Normal(0, 1) 
\\ \tau &\sim HalfCauchy(0, .5) 


\end{align*}


In [9]:
# Generate pymc Bayesian Hierachial model
with pm.Model() as model:
    
    intercept = pm.Normal("intercept", mu = 0, sigma = 1)
    sigma_a = pm.HalfCauchy("sigma_a", beta=.5)
    # Random intercepts
    alpha = pm.Normal('alpha', mu=0, sigma = sigma_a, shape=len(df["Author"].unique()))
    y = pm.Normal("y", mu = intercept + alpha[author_id], sigma = seTE, observed=TE)
   

In [10]:
# Compile nutpie pymc model.
compiled_model = nutpie.compile_pymc_model(model)
trace = nutpie.sample(compiled_model)

Progress,Draws,Divergences,Step Size,Gradients/Draw
,1300,0,0.62,7
,1300,0,0.57,7
,1300,0,0.58,7
,1300,0,0.62,7
,1300,0,0.57,11
,1300,0,0.61,15


In [13]:
# Get the results of the model MCMC samples.
az.summary(trace, var_names=["sigma_a", "intercept"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
sigma_a,0.293,0.099,0.118,0.483,0.003,0.002,1213.0,922.0,1.0
intercept,0.569,0.094,0.390,0.745,0.002,0.001,2125.0,2432.0,1.0


# References
Harrer, M., Cuijpers, P., Furukawa, T.A., & Ebert, D.D. (2021). Doing Meta-Analysis with R: A Hands-On Guide. Boca Raton, FL and London: Chapmann & Hall/CRC Press. ISBN 978-0-367-61007-4.

Röver, Christian. 2017. “Bayesian Random-Effects Meta-Analysis Using the ‘Bayesmeta‘ r Package.” ArXiv Preprint 1711.08683.